In [1]:
import cv2  
import numpy as np
from time import time
import torch

In [2]:
class wasteDetection:
    """

    """


    def __init__(self, capture_url, model_name):
        """


        """
        self.capture_index = capture_url
        self.model = self.load_model(model_name)
        self.classes = self.model.names
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.pixel_cm_ratio = 1
        # Load Aruco detector
        self.parameters = cv2.aruco.DetectorParameters_create()
        self.aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_7X7_250)
        self.aruco_in_frame = False
        print("Using Device: ", self.device)


    def get_video_capture(self): 
        """

        """
        # self.capture_index
        return cv2.VideoCapture(self.capture_index)


    def load_model(self, model_name): 
        """

        """
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_name, force_reload=True)
        
        return model


    def score_frame(self, frame):
        """

        """

        self.model.to(self.device)
        corners, _, _ = cv2.aruco.detectMarkers(frame, self.aruco_dict, parameters=self.parameters)
        #frame = [frame]
        results = self.model([frame])
        labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
        # Get Aruco marker
        
        if corners:
            # Draw polygon around the marker
            
            int_corners = np.int0(corners)
            corners_vect = np.array(corners[0])
            rect = cv2.minAreaRect(corners_vect)
            (x, y), (w, h), angle = rect
            cv2.polylines(frame, int_corners, True, (0, 255, 0), 5)

            # Aruco Perimeter
            aruco_perimeter = cv2.arcLength(corners[0], True)
            # Pixel to cm ratio
            self.pixel_cm_ratio = aruco_perimeter / 20
            self.aruco_in_frame = True
            
        else: 
            self.aruco_in_frame = False
            
            
        return labels, cord

 
    def class_to_label(self, x): 
        """

        """

        return self.classes[int(x)]


    def plot_boxes(self, results, frame):
        """

        """
        
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        
        for i in range(n):
            
            row = cord[i]
            x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape) 
            vect_2d = np.array([[x1, y1], [x1, y2], [x2, y1], [x2, y2]]) # make a 2d vector for the points
            rect = cv2.minAreaRect(vect_2d)
            (x, y), (w, h), angle = rect
            
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)
            cv2.polylines(frame, [box], True, (255, 0, 0), 2)
            cv2.putText(frame, self.class_to_label(labels[i]), (x2, y2), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            
            # Get Width and Height of the Objects by applying the Ratio pixel to cm
            if self.aruco_in_frame:
                
                object_width = w / self.pixel_cm_ratio
                object_height = h / self.pixel_cm_ratio
                
                
                cv2.putText(frame, "Width {} cm".format(round(object_width, 1)), (int(x - 100), int(y - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)
                cv2.putText(frame, "Height {} cm".format(round(object_height, 1)), (int(x - 100), int(y + 15)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)
            
                
        
        return frame
    
    
    def __call__(self):
        """

        """

        
        cap = self.get_video_capture()
        
        #assert cap.isOpened()

        while True:
            ret, frame = cap.read()
            assert ret
            #504, 378
            frame = cv2.resize(frame, (640,640))

            start_time = time()
            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)
            end_time = time()
            
            
            fps = 1/np.round(end_time - start_time, 2)

            cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

            cv2.imshow('YOLOv5 Detection', frame)

            if cv2.waitKey(5) & 0xFF == 27:
                break

        cap.release()

In [3]:
detector = wasteDetection(capture_url='https://192.168.1.197:8080/video', model_name='best_5.pt')
detector()

#'https://192.168.201.208:8080/video'

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\farza/.cache\torch\hub\master.zip
YOLOv5  2022-6-16 Python-3.8.8 torch-1.8.2 CPU

Fusing layers... 
Model summary: 213 layers, 7026307 parameters, 0 gradients
Adding AutoShape... 


Using Device:  cpu


AssertionError: 